In [1]:
import polars as pl

In [10]:
train_df = pl.read_csv("../fold/train_2019_with_fold_5.csv")
print(train_df.shape)

(73163, 31)


In [13]:
train_df.select("target").null_count()

target
u32
0


In [11]:
((train_df.select("tbp_lv_H") + 2) / 110).min()

tbp_lv_H
f64
0.003871


In [18]:
# targetが1の患者のpatient_idを取得
target_one_patient_ids = train_df.filter(pl.col("target") == 1)["patient_id"].unique()

# 対象のpatient_idを持つ行のみをフィルタリング
train_df = train_df.filter(pl.col("patient_id").is_in(target_one_patient_ids))

In [19]:
train_df.shape

(156004, 57)

In [20]:
train_df = train_df.with_columns(
    (pl.col("lesion_id").is_not_null().cast(pl.Int8)).alias("has_lesion_id")
)

In [21]:
train_df["has_lesion_id"].describe()

statistic,value
str,f64
"""count""",156004.0
"""null_count""",0.0
"""mean""",0.03489
"""std""",0.183502
"""min""",0.0
"""25%""",0.0
"""50%""",0.0
"""75%""",0.0
"""max""",1.0


In [7]:
train_df["target"].describe()

statistic,value
str,f64
"""count""",156004.0
"""null_count""",0.0
"""mean""",0.002519
"""std""",0.050128
"""min""",0.0
"""25%""",0.0
"""50%""",0.0
"""75%""",0.0
"""max""",1.0


In [9]:
import pandas as pd

In [10]:
train_df2 = pd.read_csv("./fold/train_with_fold_4.csv")
print(train_df2.shape)

/tmp/ipykernel_164592/764784266.py:1: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df2 = pd.read_csv("./fold/train_with_fold_4.csv")


(401059, 56)


In [22]:
train_df2["has_lesion_id"] = train_df2["lesion_id"].notnull().astype(int)

In [23]:
train_df2["has_lesion_id"]

0         0
1         1
2         0
3         0
4         0
         ..
401054    1
401055    0
401056    1
401057    0
401058    0
Name: has_lesion_id, Length: 401059, dtype: int64

In [1]:
import timm

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import timm

# 利用可能なモデルのリストを取得
model_names = timm.list_models()

# ConvNeXt V2 系モデルをフィルタリング
convnextv2_models = [name for name in model_names if 'convnext' in name]

# 利用可能な ConvNeXt V2 系モデルを表示
print(convnextv2_models)

['convnext_atto', 'convnext_atto_ols', 'convnext_base', 'convnext_femto', 'convnext_femto_ols', 'convnext_large', 'convnext_large_mlp', 'convnext_nano', 'convnext_nano_ols', 'convnext_pico', 'convnext_pico_ols', 'convnext_small', 'convnext_tiny', 'convnext_tiny_hnf', 'convnext_xlarge', 'convnext_xxlarge', 'convnextv2_atto', 'convnextv2_base', 'convnextv2_femto', 'convnextv2_huge', 'convnextv2_large', 'convnextv2_nano', 'convnextv2_pico', 'convnextv2_small', 'convnextv2_tiny']


In [14]:
import timm

# ConvNeXt Small モデルの事前学習タグを取得
model_name = 'convnextv2_base'
pretrained_tags = timm.list_pretrained(model_name)

print(f"Pretrained tags for {model_name}:")
for tag in pretrained_tags:
    print(tag)

Pretrained tags for convnextv2_base:
convnextv2_base.fcmae
convnextv2_base.fcmae_ft_in1k
convnextv2_base.fcmae_ft_in22k_in1k
convnextv2_base.fcmae_ft_in22k_in1k_384


In [9]:
timm.create_model('convnextv2_small')

ConvNeXt(
  (stem): Sequential(
    (0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
    (1): LayerNorm2d((96,), eps=1e-06, elementwise_affine=True)
  )
  (stages): Sequential(
    (0): ConvNeXtStage(
      (downsample): Identity()
      (blocks): Sequential(
        (0): ConvNeXtBlock(
          (conv_dw): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
          (norm): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
          (mlp): GlobalResponseNormMlp(
            (fc1): Linear(in_features=96, out_features=384, bias=True)
            (act): GELU()
            (drop1): Dropout(p=0.0, inplace=False)
            (grn): GlobalResponseNorm()
            (fc2): Linear(in_features=384, out_features=96, bias=True)
            (drop2): Dropout(p=0.0, inplace=False)
          )
          (shortcut): Identity()
          (drop_path): Identity()
        )
        (1): ConvNeXtBlock(
          (conv_dw): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), 

In [1]:
import pandas as pd

In [5]:
df_past = pd.read_csv("../data/metadata.csv", low_memory=False)
df_new = pd.read_csv("../data/train-metadata.csv", low_memory=False)

In [6]:
df_past.shape

(81722, 28)

In [7]:
df_new.shape

(401059, 55)

In [53]:
df_past["copyright_license"].value_counts()

copyright_license
CC-BY-NC    49723
CC-0        20086
CC-BY       11913
Name: count, dtype: int64

In [8]:
"""

        self.df["age_scaled"] = (
            self.df["age_approx"].fillna(60) / 90
        )  # 後でloglossで評価するのでクソでかい数字で割っておく
    self.df["sex"] = self.df["sex"].fillna("male")
    self.df["anatom_site_general"] = self.df["anatom_site_general"].fillna("unk")
    self.df["has_lesion_id"] = self.df["lesion_id"].notnull().astype(int)
    # 最小値が-1.5くらい、最大値が105くらいなので[0, 1]に収まるように
    self.df["tbp_lv_H"] = (self.df["tbp_lv_H"] + 2) / 110

    # 0fill
    self.df = self.df.infer_objects()  # これやっとかないと警告が出る
    self.df = self.df.fillna(0)
    self.df["anatom_site_general_enc"] = self.df["anatom_site_general"].map(
        ANATOM_SITE_GENERAL_ENCODER
    )
    self.df["sex_enc"] = self.df["sex"].map(SEX_ENCODER)
    """

'\n\n        self.df["age_scaled"] = (\n            self.df["age_approx"].fillna(60) / 90\n        )  # 後でloglossで評価するのでクソでかい数字で割っておく\n    self.df["sex"] = self.df["sex"].fillna("male")\n    self.df["anatom_site_general"] = self.df["anatom_site_general"].fillna("unk")\n    self.df["has_lesion_id"] = self.df["lesion_id"].notnull().astype(int)\n    # 最小値が-1.5くらい、最大値が105くらいなので[0, 1]に収まるように\n    self.df["tbp_lv_H"] = (self.df["tbp_lv_H"] + 2) / 110\n\n    # 0fill\n    self.df = self.df.infer_objects()  # これやっとかないと警告が出る\n    self.df = self.df.fillna(0)\n    self.df["anatom_site_general_enc"] = self.df["anatom_site_general"].map(\n        ANATOM_SITE_GENERAL_ENCODER\n    )\n    self.df["sex_enc"] = self.df["sex"].map(SEX_ENCODER)\n    '

In [10]:
print(df_past["age_approx"].describe())
print(df_new["age_approx"].describe())

count    79702.000000
mean        47.849489
std         19.865311
min          0.000000
25%         35.000000
50%         50.000000
75%         65.000000
max         85.000000
Name: age_approx, dtype: float64
count    398261.000000
mean         58.012986
std          13.596165
min           5.000000
25%          50.000000
50%          60.000000
75%          70.000000
max          85.000000
Name: age_approx, dtype: float64


In [19]:
print(df_past["sex"].value_counts(normalize=True))
print(df_new["sex"].value_counts(normalize=True))

sex
male      0.538947
female    0.461053
Name: proportion, dtype: float64
sex
male      0.681688
female    0.318312
Name: proportion, dtype: float64


In [18]:
print(df_past["anatom_site_general"].value_counts(normalize=True))
print(df_new["anatom_site_general"].value_counts(normalize=True))

anatom_site_general
lower extremity    0.249803
anterior torso     0.216224
posterior torso    0.210664
upper extremity    0.155275
head/neck          0.135922
lateral torso      0.016002
palms/soles        0.013052
oral/genital       0.003058
Name: proportion, dtype: float64
anatom_site_general
posterior torso    0.308376
lower extremity    0.260630
anterior torso     0.222032
upper extremity    0.178488
head/neck          0.030473
Name: proportion, dtype: float64


In [22]:
print(df_past["lesion_id"].isnull().mean())
print(df_new["lesion_id"].isnull().mean())

0.2172731944886322
0.945000610882688


In [25]:
#print(df_past["tbp_lv_H"].isnull().mean())
print(df_new["tbp_lv_H"].mean())

54.65368914353473


In [28]:
df = pd.concat([df_new, df_past], axis=0)

In [29]:
len(df_new), len(df_past), len(df)

(401059, 81722, 482781)

In [32]:
df_past["benign_malignant"].value_counts()

benign_malignant
benign                     64047
malignant                   9239
indeterminate                150
indeterminate/malignant       85
indeterminate/benign          67
Name: count, dtype: int64

In [36]:
df_past["has_lesion_id"] = df_past["lesion_id"].notnull().astype(int)
df_new["has_lesion_id"] = df_new["lesion_id"].notnull().astype(int)

In [49]:
dup = pd.read_csv("../data/2020_Challenge_duplicates.csv")

In [50]:
dup

,ISIC_id,ISIC_id_paired,partition
0,ISIC_0079038,ISIC_8521950,train
1,ISIC_0087297,ISIC_4755972,train
2,ISIC_0088137,ISIC_4201955,train
3,ISIC_0112097,ISIC_5934021,train
4,ISIC_0148783,ISIC_7460560,train
...,...,...,...
527,ISIC_8987085,ISIC_9751079,train
528,ISIC_9008728,ISIC_9982577,test
529,ISIC_9218360,ISIC_9913406,train
530,ISIC_9432244,ISIC_9645937,train


In [52]:
df_past["patient_id"].isnull().sum()

40594

In [48]:
df_past[df_past["benign_malignant"].isnull()]

,isic_id,attribution,copyright_license,acquisition_day,age_approx,anatom_site_general,benign_malignant,clin_size_long_diam_mm,concomitant_biopsy,dermoscopic_type,...,mel_type,mel_ulcer,melanocytic,nevus_type,patient_id,personal_hx_mm,pixels_x,pixels_y,sex,has_lesion_id
3116,ISIC_8780653,Konstantinos Liopyris,CC-BY,NaN,35.0,lower extremity,NaN,NaN,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4032,3024,female,0
3117,ISIC_8572047,University of Pittsburgh Medical Center and Hi...,CC-BY,0.0,40.0,posterior torso,NaN,NaN,False,NaN,...,NaN,NaN,NaN,NaN,IP_5371923,NaN,4500,3000,male,0
3118,ISIC_4416957,University of Pittsburgh Medical Center and Hi...,CC-BY,1067.0,40.0,posterior torso,NaN,NaN,False,NaN,...,NaN,NaN,NaN,NaN,IP_5371923,NaN,4256,2832,male,0
3119,ISIC_4588100,University of Pittsburgh Medical Center and Hi...,CC-BY,0.0,50.0,posterior torso,NaN,NaN,False,NaN,...,NaN,NaN,NaN,NaN,IP_6058143,NaN,4500,3000,female,0
3120,ISIC_5296938,University of Pittsburgh Medical Center and Hi...,CC-BY,1337.0,55.0,posterior torso,NaN,NaN,False,NaN,...,NaN,NaN,NaN,NaN,IP_6058143,NaN,4500,3000,female,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70277,ISIC_0011444,Anonymous,CC-0,NaN,85.0,upper extremity,NaN,NaN,False,contact non-polarized,...,NaN,NaN,NaN,NaN,IP_4977118,NaN,6648,4419,male,1
70402,ISIC_0011319,Anonymous,CC-0,NaN,30.0,posterior torso,NaN,7.0,False,NaN,...,NaN,NaN,True,NaN,NaN,False,1024,768,female,0
71257,ISIC_0010464,Anonymous,CC-0,NaN,30.0,upper extremity,NaN,5.0,False,NaN,...,NaN,NaN,True,NaN,NaN,False,1024,768,male,0
71439,ISIC_0010282,Anonymous,CC-0,NaN,85.0,head/neck,NaN,8.0,False,NaN,...,NaN,NaN,True,NaN,NaN,True,1024,768,male,0


In [47]:
df_past["benign_malignant"].map(
    {
        "benign": 0,
        "malignant": 1,
        "indeterminate": 0,
        "indeterminate/malignant": 1,
        "indeterminate/benign": 0
    }
).isnull().sum()

8134

In [35]:
pd.DataFrame(df_past[["benign_malignant", "has_lesion_id"]].value_counts())

count
benign_malignant        has_lesion_id       
benign                  1              48789
                        0              15258
malignant               1               6926
                        0               2313
indeterminate           0                 84
indeterminate/malignant 1                 80
indeterminate           1                 66
indeterminate/benign    1                 66
indeterminate/malignant 0                  5
indeterminate/benign    0                  1

In [38]:
pd.DataFrame(df_new[["target", "has_lesion_id"]].value_counts())

count
target has_lesion_id        
0      0              379001
       1               21665
1      1                 393

In [41]:
len(set(df_past["lesion_id"]) - set(df_new["lesion_id"]))

48490

In [42]:
len(set(df_new["lesion_id"]) - set(df_past["lesion_id"]))

20470

In [43]:
len(set(df_new["lesion_id"])), len(set(df_past["lesion_id"])), 

(22059, 50079)

In [14]:
import glob
isic_ids2category = {}
for file in sorted(glob.glob("../past_metadata/isic*.csv")):
    df = pd.read_csv(file, low_memory=False)
    category = file.split("isic_meta_")[-1].split(".csv")[0]
    print(f"{category} len {len(df)}")
    for _isic_id in df["isic_id"].values:
        if _isic_id in isic_ids2category:
            # デバッグ用
            #print(_isic_id, category, isic_ids2category[_isic_id])
            isic_ids2category[_isic_id] = category
            pass
        else:
            isic_ids2category[_isic_id] = category

challenge_2016_test len 379
challenge_2016_training len 900
challenge_2017_test len 600
challenge_2017_training len 2000
challenge_2017_validation len 150
challenge_2018_task_1-2_test len 1000
challenge_2018_task_1-2_training len 2594
challenge_2018_task_1-2_validation len 100
challenge_2018_task_3_test len 1512
challenge_2018_task_3_training len 10015
challenge_2018_task_3_validation len 193
challenge_2019_training len 25331
challenge_2020_training len 33126


In [16]:
df_past = pd.read_csv("../data/metadata.csv", low_memory=False)
print(df_past.shape)
df_past["isic_challenge_category"] = df_past["isic_id"].map(isic_ids2category)
dup = pd.read_csv("../data/2020_Challenge_duplicates.csv")
df_past = df_past.merge(dup[["ISIC_id", "ISIC_id_paired"]], how="left", left_on="isic_id", right_on="ISIC_id")
df_past = df_past[df_past["ISIC_id_paired"].isnull()].reset_index(drop=True)
del df_past["ISIC_id_paired"]
print(df_past.shape)

(81722, 28)
(81297, 30)


In [17]:
df_past["isic_challenge_category"].value_counts()

isic_challenge_category
challenge_2020_training               32701
challenge_2019_training               25331
challenge_2018_task_3_test             1512
challenge_2018_task_1-2_test           1000
challenge_2018_task_3_validation        193
challenge_2018_task_1-2_validation      100
challenge_2016_training                  41
challenge_2016_test                      26
challenge_2017_test                       4
challenge_2018_task_1-2_training          1
Name: count, dtype: int64

In [ ]:
df_new = pd.read_csv("../data/train-metadata.csv", low_memory=False)

In [22]:

for i in df_past["isic_challenge_category"].unique():
    df_past1 = df_past[df_past["isic_challenge_category"] == i]
    print("==================================")
    print(i)
    print("age ----")
    print(df_new["age_approx"].describe())
    print(df_past1["age_approx"].describe())
    print("")
    print("site ----")
    print(df_new["anatom_site_general"].value_counts(normalize=True))
    print(df_past1["anatom_site_general"].value_counts(normalize=True))
    print("")
    print("lesion ----")
    print(df_new["lesion_id"].isnull().mean())
    print(df_past1["lesion_id"].isnull().mean())
    print(len(set(df_new["lesion_id"]) & set(df_past1["lesion_id"])) / (len(set(df_past1["lesion_id"])) + 1))
    print("")
    print("target ----")
    print(df_past1["benign_malignant"].isnull().mean())
    print(df_past1["benign_malignant"].value_counts())
    print("patient ----")
    print(df_past1["patient_id"].isnull().mean())

nan
age ----
count    398261.000000
mean         58.012986
std          13.596165
min           5.000000
25%          50.000000
50%          60.000000
75%          70.000000
max          85.000000
Name: age_approx, dtype: float64
count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: age_approx, dtype: float64

site ----
anatom_site_general
posterior torso    0.308376
lower extremity    0.260630
anterior torso     0.222032
upper extremity    0.178488
head/neck          0.030473
Name: proportion, dtype: float64
Series([], Name: proportion, dtype: float64)

lesion ----
0.945000610882688
nan
0.0

target ----
nan
Series([], Name: count, dtype: int64)
patient ----
nan
challenge_2020_training
age ----
count    398261.000000
mean         58.012986
std          13.596165
min           5.000000
25%          50.000000
50%          60.000000
75%          70.000000
max          85.000000
Name: age_approx, dtype: float64
count    32657.000000
m

In [2]:
import h5py
from io import BytesIO
import numpy as np
import cv2
import pandas as pd
import tqdm
past_fp_hdf = h5py.File("../data/image_256sq.hdf5", mode="r")

In [52]:
past_df = pd.read_csv("../fold/train_2019_with_fold_5.csv")
for image_id_past in tqdm.tqdm(past_df["isic_id"].values):
    image_data = past_fp_hdf[image_id_past][()]
    image_stream = BytesIO(image_data)
    # BytesIOからバイトデータを取得
    byte_array = np.asarray(bytearray(image_stream.read()), dtype=np.uint8)
    # OpenCVで画像データをデコード
    image = cv2.imdecode(byte_array, cv2.IMREAD_COLOR)
    # NaNまたは無限大が含まれているか確認
    contains_nan_or_inf = np.isnan(image).any() or np.isinf(image).any()
    if contains_nan_or_inf:
        print(f"Contains NaN or inf: {contains_nan_or_inf}, {image_id_past}")

100%|██████████| 17784/17784 [00:16<00:00, 1056.24it/s]


In [53]:
past_df = pd.read_csv("../fold/train_2020_with_fold_5.csv")
for image_id_past in tqdm.tqdm(past_df["isic_id"].values):
    image_data = past_fp_hdf[image_id_past][()]
    image_stream = BytesIO(image_data)
    # BytesIOからバイトデータを取得
    byte_array = np.asarray(bytearray(image_stream.read()), dtype=np.uint8)
    # OpenCVで画像データをデコード
    image = cv2.imdecode(byte_array, cv2.IMREAD_COLOR)
    # NaNまたは無限大が含まれているか確認
    contains_nan_or_inf = np.isnan(image).any() or np.isinf(image).any()
    if contains_nan_or_inf:
        print(f"Contains NaN or inf: {contains_nan_or_inf}, {image_id_past}")

100%|██████████| 32701/32701 [00:32<00:00, 1003.48it/s]


In [33]:
fp_hdf = h5py.File("../data/train-image.hdf5", mode="r")

In [41]:
image_id = "ISIC_0051648"
image_data = fp_hdf[image_id][()]
image_stream = BytesIO(image_data)
# BytesIOからバイトデータを取得
byte_array = np.asarray(bytearray(image_stream.read()), dtype=np.uint8)
# OpenCVで画像データをデコード
image = cv2.imdecode(byte_array, cv2.IMREAD_COLOR)
image.min()

27

In [5]:
past_df = pd.read_csv("../fold/train_2020_with_fold_5.csv")
new_df = pd.read_csv("../fold/train_with_fold_5.csv")
new_df[new_df["isic_id"] =="ISIC_2597185"]

/tmp/ipykernel_27909/3568144591.py:2: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  new_df = pd.read_csv("../fold/train_with_fold_5.csv")


,isic_id,target,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,...,iddx_full,iddx_1,iddx_2,iddx_3,iddx_4,iddx_5,mel_mitotic_index,mel_thick_mm,tbp_lv_dnn_lesion_confidence,fold
101900,ISIC_2597185,0,IP_8457418,65.0,female,upper extremity,3.12,TBP tile: close-up,3D: XP,20.47888,...,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,97.62937,4


In [6]:
past_df[past_df["isic_id"] == "ISIC_2597185"]

,isic_id,attribution,copyright_license,acquisition_day,age_approx,anatom_site_general,benign_malignant,clin_size_long_diam_mm,concomitant_biopsy,dermoscopic_type,...,melanocytic,nevus_type,patient_id,personal_hx_mm,pixels_x,pixels_y,sex,isic_challenge_category,target,fold
